In [7]:
from preprocess_functions import load_cleaned_data
FILEPATH = 'data/gtd.zip'

In [8]:
import zipfile
csv_file = zipfile.ZipFile(FILEPATH, 'r')

In [9]:
csv_file.read('globalterrorismdb_0617dist.csv')

KeyError: "There is no item named 'globalterrorismdb_0617.csv' in the archive"